Deep learning -   HW-1

Based on the article :Flowers classification using PyTorch
By - Michele Di Fazio

https://blog.jovian.ai/flowers-classification-using-pytorch-5c30bc8e559c




Purpose :
Implementation of an algorithm that knows how to make a classification for flower types.
Database - Public databases of flowers



In [ ]:
import os
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torchvision.transforms as transforms
from torchvision.datasets.folder import default_loader
%matplotlib inline

flowers_idx -Data base
first Col=ID secend Col =flower’s specie

In [ ]:
# idx = pd.read_csv("../input/flower-goggle-tpu-classification/flowers_idx.csv")
# idx

Base modle
1.coumpute the loss func 
2. adjust the weight by  gradient descend
3. compere between the modles and chosse


Base modle
1.coumpute the loss func 
2. adjust the weight by  gradient descend
3. compere between the modles and pick the best 

build custom orgianse dataBase with labels use dataset 


input- cvs_file ,path, transform   


code ->build calss of dataset

In [ ]:
class MyTestdata(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.label = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        image = os.path.join(self.root_dir, f"{self.label.iloc[index, 0]}.jpeg")
        image = default_loader(image)
        label = self.label.iloc[index, 1]
        
        if self.transform:
            image = self.transform(image)
            
        return (image, label)

Transform Code - Kind of validation for the data Base that the images is in the right standard . 

In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor()
])

test_ds = MyTestdata(
    csv_file="../input/flower-goggle-tpu-classification/flowers_idx.csv",
    root_dir="../input/flower-goggle-tpu-classification/flower_tpu/flower_tpu/flowers_google/flowers_google",
    transform=transform_test
)

FileNotFoundError: ignored

Add the other Two data base - 


1.104 Flowers: Garden of Eden


2.  Flower google tpu classification

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(128, padding=4, padding_mode="reflect"),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
    #transforms.Normalize([.5, .5, .5], [.5, .5, .5])
])
    
train_ds = torchvision.datasets.ImageFolder(
    root="../input/104-flowers-garden-of-eden/jpeg-192x192/train",
    transform=transform_train
)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(128, padding=4, padding_mode="reflect"),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
    #transforms.Normalize([.5, .5, .5], [.5, .5, .5])
])
    
train_ds = torchvision.datasets.ImageFolder(
    root="../input/104-flowers-garden-of-eden/jpeg-192x192/train",
    transform=transform_train
)

DataLoader= help to arrange the data after the transforming .
batch_sise = anount of input . 

shuffle=true mix the images in the class. 

In [ ]:
batch_size = 128 
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, num_workers=3, pin_memory=True)

Simulator gives a view based on the data.

In [ ]:
def show_batch(train_dl):
    for images, labels in train_dl:
        fig, ax = plt.subplots(figsize=(16,16))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:64], nrow=8).permute(1,2,0))
        break

bulid ImageClassificationBase class sub class help to clac the loss func and train the modal  

accuracy =numerical value that helps to assimilate how much the model is accurate.

training_step = calc the loss function.
which in this case will be the **cross-entropy loss**( special type of loss function)



In [ ]:


def accuracy(out, labels):
    _, preds = torch.max(out, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {"val_loss": loss.detach(), "val_acc": acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_acc = [x["val_acc"] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"val_loss": epoch_loss.item(), "val_acc": epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
            print("Epoch: [{}], last_lr: {:.6f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
                epoch, result["lrs"][-1], result["train_loss"], result["val_loss"], result["val_acc"]))


convolution=operation whereby a kernel, The structure of the model

scan the image - by slides over the image collact the information from a few images into a "limited space". 
this type of structure help us Define W The model as a unique model,  store more information about the image.




In [ ]:
resnet18 = models.resnet18()
resnet18


in this moddle uses resent 34 .



Definition of three mechanisms that help us control the running of the algorithm:



  forward- Running and self-learning

freeze- Reboots the parameters, will help to refine the algorithm later.

unfreeze- Set the parameters as good parameters.

In [ ]:
class flowers_resnet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = models.resnet34(pretrained=True)
        number_features = self.network.fc.in_features
        self.network.fc = nn.Linear(number_features, 104)
        
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.fc.parameters():
            param.require_grad = True
            
    def unfreeze(self):
        for param in self.network.parameters():
            param.require_grad = True

Use of colab gpu.


 The explanation is that the program needs a processor with many cores to perform a calculation at the same time if running on the PC (without gpu) could lead to the crash of the computer.

In [ ]:
def select_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for x in self.dl:
            yield to_device(x, self.device)
            
    def __len__(self):
        return len(self.dl)

check if GPU is  available

In [ ]:
device = select_device()
device

NameError: ignored

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

last part of the moddle will define method to evaluate the modal in each epoch(loop) complate. 




In [ ]:
@torch.no_grad()
def evaluate(model, val_dl):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]
    
def fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, weight_decay=0, 
                  grad_clip=None, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                               steps_per_epoch=len(train_dl))
    
    for epoch in range(epochs):
        model.train()
        train_loss = []
        lrs = []
        for batch in tqdm(train_dl):
            loss = model.training_step(batch)
            train_loss.append(loss)
            loss.backward()
            
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            sched.step()
            
        result = evaluate(model, val_dl)
        result["train_loss"] = torch.stack(train_loss).mean().item()
        result["lrs"] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model = to_device(flowers_resnet(), device)

Getting  info about the modle 


In [ ]:
history = [evaluate(model, val_dl)]
history

model.freeze()

In [ ]:
model.freeze()

# ***One of the most important parts of the model***

by editing this part we can improve our module for better result- 

To answer the question of how to improve the algorithm we based our answer on 
the following article -Understanding Learning Rates and How It Improves Performance in Deep Learning

https://towardsdatascience.com/understanding-learning-rates-and-how-it-improves-performance-in-deep-learning-d0d4059c1c10



epochs - one of the heyper parameterthat define the number of times that the algorithm will "run" on the entire dataset.
**A larger number of epochs can improve the model and produce better results, but on the other hand can extend the model time greatly and is not at all sure that it will improve the model..**

-- To answer on 



max_lr =learining rate=  another hyperparameter that controls how much influencing the weight respect on the loss gradient. 

lower value slower =low leraning rate meanis There will be a smaller chance of missing a  point in the modelcan help the  be more accurate,
If so a low rate can extend our model learning time significantly.


grad_clip - Resets the gradient vector, making the model more "flexible"


weight_decay- How important is W.


opt_func - Is the direction of progress to the maximum optimum.




In [ ]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl,
                         grad_clip=grad_clip, weight_decay=weight_decay, 
                         opt_func=opt_func)

In [ ]:
model.unfreeze()

In [ ]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl,
                         grad_clip=grad_clip, weight_decay=weight_decay, 
                         opt_func=opt_func)

Train the model until reached 86% Expected to take about 10 minutes

present the performance of the model with plots 

In [ ]:
def plot_loss(histroy):
    val_loss = [x["val_loss"] for x in history]
    train_loss = [x.get("train_loss") for x in history]
    plt.plot(val_loss, "-rx")
    plt.plot(train_loss, "-bx")
    plt.title("Loss vs. number of epochs")
    plt.legend(["Validation loss", "Training loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

plot_loss(history)

In [ ]:
def plot_accuracy(history):
    accuracy = [x["val_acc"] for x in history]
    plt.plot(accuracy, "-x")
    plt.title("Accuracy vs. number of epochs")
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    
plot_accuracy(history)

move the images of the test dataset to the GPU, and to pass them through our model

In [ ]:
def prediction(images, model):
    xb = to_device(images.unsqueeze(0), device)
    out = model(xb)
    _, preds = torch.max(out, dim=1)
    return train_ds.classes[preds[0].item()]

image 1 

In [ ]:
images, labels = test_ds[100]
plt.imshow(images.permute(1,2,0))
print("Label: ", labels, "Prediction:", prediction(images, model))

image 2

In [ ]:
images, labels = test_ds[200]
plt.imshow(images.permute(1,2,0))
print("Label: ", labels, "Prediction:", prediction(images, model))